In [ ]:
import pandas as pd
import numpy as np
import matplotlib as plt
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from evidently import Report
from evidently.presets import DataDriftPreset
import pandas as pd
import os
import mlflow
import mlflow.sklearn
from fastapi import FastAPI
import warnings
warnings.filterwarnings('ignore')

In [62]:
df = pd.read_csv('C:/Users/gito2/Downloads/house_price_prediction/Artifacts/sample_submission.csv')
df

,Id,SalePrice
0,1461,169277.052498
1,1462,187758.393989
2,1463,183583.683570
3,1464,179317.477511
4,1465,150730.079977
...,...,...
1454,2915,167081.220949
1455,2916,164788.778231
1456,2917,219222.423400
1457,2918,184924.279659


In [63]:
categorical_cols = df.select_dtypes(include=['object']).columns
le = LabelEncoder()

for col in categorical_cols:
    df[col] = le.fit_transform(df[col])

df

,Id,SalePrice
0,1461,169277.052498
1,1462,187758.393989
2,1463,183583.683570
3,1464,179317.477511
4,1465,150730.079977
...,...,...
1454,2915,167081.220949
1455,2916,164788.778231
1456,2917,219222.423400
1457,2918,184924.279659


In [64]:
df_train = pd.read_csv('C:/Users/gito2/Downloads/house_price_prediction/Artifacts/train.csv')
df_test = pd.read_csv('C:/Users/gito2/Downloads/house_price_prediction/Artifacts/test.csv')
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [65]:
# Cek jumlah missing values di setiap kolom
missing_values = df.isnull().sum()

# Filter hanya kolom yang punya missing values
missing_values = missing_values[missing_values > 0].sort_values(ascending=False)

print(missing_values)

Series([], dtype: int64)


In [66]:
df_train["LotFrontage"] = df_train.groupby("Neighborhood")["LotFrontage"].transform(
    lambda x: x.fillna(x.median())
)
df_test["LotFrontage"] = df_test.groupby("Neighborhood")["LotFrontage"].transform(
    lambda x: x.fillna(x.median())
)

In [67]:
# Split features and target
# Split dari train.csv menjadi train & validation
from sklearn.model_selection import train_test_split

X = df_train.drop(columns=['SalePrice'])
y = df_train['SalePrice']

categorical_cols = X.select_dtypes(include=["object"]).columns
numeric_cols = X.select_dtypes(exclude=["object"]).columns

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

In [68]:
numeric_features = X_train.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X_train.select_dtypes(include=['object']).columns

In [69]:
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# Preprocessor
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median"))
])
categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features)
    ]
)

In [70]:
# Pipeline final (preprocessing + model)
pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("model", RandomForestRegressor(random_state=42))
])

# Training
pipeline.fit(X_train, y_train)

,steps,"[('preprocessor', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('num', ...), ('cat', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [71]:
def generate_data_drift_report(X_train, X_test):
    report = Report(metrics=[DataDriftPreset()])
    report.run(reference_data=X_train, current_data=X_test)
    report.save_html("monitoring/data_drift_report.html")

In [72]:
param_grid = {
    "model__n_estimators": [100, 200],
    "model__max_depth": [None, 10]
}


In [73]:
# Evaluasi
# ===================
import mlflow
import mlflow.sklearn
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor



# GridSearch
grid_search = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    cv=3,
    scoring="neg_root_mean_squared_error",
    n_jobs=-1
)
mlflow.set_tracking_uri("sqlite:///mlflows.db")
mlflow.set_experiment("HousePrices-RF")

with mlflow.start_run():
    grid_search.fit(X_train, y_train)

    # Prediksi
    y_pred = grid_search.predict(X_valid)

    # Metrics
    rmse = np.sqrt(mean_squared_error(y_valid, y_pred))
    mae = mean_absolute_error(y_valid, y_pred)
    r2 = r2_score(y_valid, y_pred)

    # Logging params & metrics
    mlflow.log_params(grid_search.best_params_)
    mlflow.log_metric("RMSE", rmse)
    mlflow.log_metric("MAE", mae)
    mlflow.log_metric("R2", r2)

    # (Optional) Simpan model sebagai artifact
    mlflow.sklearn.log_model(
        sk_model=grid_search.best_estimator_,
        artifact_path="random_forest_model",
        registered_model_name="HousePrice_RandomForest"
    )



2025/08/12 14:32:12 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/08/12 14:32:21 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'HousePrice_RandomForest' already exists. Creating a new version of this model...
Created version '5' of model 'HousePrice_RandomForest'.


In [74]:
print("Best Params:", grid_search.best_params_)
print(f"RMSE: {rmse:.4f}, MAE: {mae:.4f}, R²: {r2:.4f}")

Best Params: {'model__max_depth': 10, 'model__n_estimators': 100}
RMSE: 28791.6022, MAE: 17721.7462, R²: 0.8919


In [75]:
import joblib

# pipeline adalah model yang sudah di-train
joblib.dump(pipeline, "model.pkl")

['model.pkl']

In [ ]:
mlflow.sklearn.log_model(pipeline, artifact_path="model")

    # === Evidently Data Drift Report ===
drift_report = Report(metrics=[DataDriftPreset()])
drift_report.run(reference_data=df_train, current_data=df_test)

report_path = "drift_report.html"

    # Upload ke MLflow sebagai artifact

mlflow.log_artifact(report_path, artifact_path="data_drift_report")

    # Bersihkan file sementara
os.remove(report_path)

print("✅ Training, logging metrics, and drift report completed!")

2025/08/12 14:43:31 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/08/12 14:43:37 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


✅ Training, logging metrics, and drift report completed!
